# Day 22 - More pathfinding, adding weights

At first this appears to be a matrix problem, but the modulus over 3 prime numbers rules that out quickly. Part two reveals that this is actually a pathfinding problem, with part 1 just setting you up with a map generator.

In [1]:
import math
from dataclasses import dataclass
from enum import Enum
from itertools import count
from heapq import heappop, heappush
from typing import Iterator, List, Tuple

M: int = 20183
X0: int = 16807
Y0: int = 48271
    
Pos = Tuple[int, int]

class Tool(Enum):
    neither = 0
    torch = 1
    climbing_gear = 2

class RegionType(Enum):
    rocky = 0, '.', Tool.climbing_gear, Tool.torch
    wet = 1, '=', Tool.climbing_gear, Tool.neither
    narrow = 2, '|', Tool.torch, Tool.neither
    
    def __new__(cls, value, char, *tools):
        instance = object.__new__(cls)
        instance._value_ = value
        instance.char = char
        instance.tools = tools
        return instance
        
    def __str__(self):
        return self.char

@dataclass(frozen=True)
class Node:
    """Node on the A* search graph"""
    x: int = 0
    y: int = 0
    tool: Tool = Tool.torch
    time: int = 0
        
    @property
    def pos(self) -> Pos:
        return self.x, self.y
        
    def cost(self, target: Pos) -> int:
        """Calculate the cost for this node, f(n) = g(n) + h(n)
        
        The cost of this node is the time taken (g) plus
        estimated cost to get to nearest goal (h).
        
        Here we use the manhattan distance to the target as
        the estimated cost.
        
        """
        return self.time + abs(target[0] - self.x) + abs(target[1] - self.y)
    
    def transitions(self, maze: 'ModMaze') -> Iterator['Node']:
        cls = type(self)
        other = next(t for t in maze[self.pos].tools if t is not self.tool)
        # switching tools
        yield cls(self.x, self.y, other, self.time + 7)
        positions = (
            (self.x + dx, self.y + dy)
            for dx, dy in ((-1, 0), (0, -1), (0, 1), (1, 0))
        )
        yield from (
            cls(x, y, self.tool, self.time + 1) for x, y in positions
            if x >= 0 and y >= 0 and self.tool in maze[x, y].tools
        )

class ModeMaze:
    def __init__(self, depth: int, target: Pos) -> None:
        self.depth = depth
        self.target = target
        # y rows, x columns
        self._erosion_levels: List[List[int]] = []
        self._width, self._height = 0, 0

    def _gen_levels(self, pos: Pos):
        # make sure there is enough erosion level info up to the given position
        # We'll grow this dynamically to the next power of 2 larger than needed, each time
        # to amortize the cost
        cw, ch = self._width, self._height
        if cw >= pos[0] + 1 and ch >= pos[1] + 1:
            return
        
        width = max(2 ** math.ceil(math.log2(pos[0] + 1)), cw)
        height = max(2 ** math.ceil(math.log2(pos[1] + 1)), ch)
                
        depth = self.depth
        maze = self._erosion_levels
        tpos = self.target
        
        # add more rows
        for y in range(ch, height):
            # "or 1" to generate the first column when empty
            above = maze[y - 1] if y else [(x * X0 + depth) % M for x in range(cw or 1)]
            row = [(y * Y0 + self.depth) % M]
            for x, p in enumerate(above[1:], 1):
                row.append(0 if (x, y) == tpos else (p * row[-1] + self.depth) % M)
            maze.append(row)
        
        # add more columns
        for x in range(cw or 1, width):
            rows = iter(maze)
            next(rows).append((x * X0 + depth) % M)
            for y, (p, row) in enumerate(zip(maze, rows), 1):
                row.append(0 if (x, y) == pos else (p[x] * row[-1] + depth) % M)
                
        self._width, self._height = width, height
            
    @property
    def risk_level(self):
        self._gen_levels(self.target)
        tx, ty = self.target
        return sum(sum(i % 3 for i in row[:tx + 1]) for row in self._erosion_levels[:ty + 1])
    
    def __getitem__(self, pos: Pos) -> RegionType:
        self._gen_levels(pos)
        return RegionType(self._erosion_levels[pos[1]][pos[0]] % 3)
    
    def __str__(self):
        lines = [[str(RegionType(i % 3)) for i in row] for row in self._erosion_levels]
        try:
            lines[0][0] = 'M'
            lines[self.target[1]][self.target[0]] = 'T'
        except IndexError:
            pass
        return '\n'.join([''.join(l) for l in lines])
    
    def rescue(self) -> int:
        start = Node()
        open = {start}
        unique = count()  # tie breaker when costs are equal
        pqueue = [(start.cost(self.target), next(unique), start)]
        closed = set()
        times = {(start.pos, start.tool): start.time}  # (pos, tool) -> time. Ignore nodes that take longer
        while open:
            node = heappop(pqueue)[-1]

            if node.pos == self.target and node.tool is Tool.torch:
                return node.time

            open.remove(node)
            closed.add(node)
            for new in node.transitions(self):
                if new in closed or new in open:
                    continue
                if times.get((new.pos, new.tool), float('inf')) < new.time:
                    continue
                times[new.pos, new.tool] = new.time
                open.add(new)
                heappush(pqueue, (new.cost(self.target), next(unique), new))

In [2]:
test = ModeMaze(510, (10, 10))
assert test.risk_level == 114
str(test) == """\
M=.|=.|.|=.|=|=.
.|=|=|||..|.=...
.==|....||=..|==
=.|....|.==.|==.
=|..==...=.|==..
=||.=.=||=|=..|=
|.=.===|||..=..|
|..==||=.|==|===
.=..===..=|.|||.
.======|||=|=.|=
.===|=|===T===||
=|||...|==..|=.|
=.=|=.=..=.||==|
||=|=...|==.=|==
|=.=||===.|||===
||.|==.|.|.||=||"""
assert test.rescue() == 45

In [3]:
import aocd
import re

data = aocd.get_data(day=22, year=2018)
depth, tx, ty = map(int, re.findall(r'\d+', data))
maze = ModeMaze(depth, (tx, ty))

In [4]:
print('Part 1:', maze.risk_level)

Part 1: 8090


In [5]:
print('Part 2:', maze.rescue())

Part 2: 980
